# 0 Set paths, devices, etc.
```
select your number below
```

In [1]:
# ======================================================
# ======================================================

my_batch_size = 1200 # 1, 30, 60, 120, 300, 600, 900
my_file = "all"  # 1, 2, 3, "all"
aggregated = False # False

# ======================================================
# ======================================================

In [2]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm

import h5py
import csv
import os

from datasets import Dataset

In [4]:
# notebook_login()

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
data_dir = 'drive/MyDrive/neuro2voc/task_1/data/'
# data_dir = '/home/zubat/Fei/Task_1/outputs/'

In [7]:
load_tensor_data_dir = data_dir+'2_tensors/data_'
load_tensor_labels_dir = data_dir+'2_tensors/labels_'
data_labels_temporal_dir = data_dir+"3_data_labels_df_temporal/"
data_labels_spatial_dir = data_dir+"3_data_labels_df_spatial/"
data_labels_aggregated_dir = data_dir+"3_data_labels_df_aggregated/"
for_tokenizer_dir = data_dir+"3_train_tokenizer/"

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

Using device: cpu



# 1 Load data and inspect

### 1.1 Load the data

In [9]:
if my_file == "all":
    if aggregated:
        tensor_1 = torch.load(f"{load_tensor_data_dir}1_{my_batch_size}_aggregated.pt")
        tensor_2 = torch.load(f"{load_tensor_data_dir}2_{my_batch_size}_aggregated.pt")
        tensor_3 = torch.load(f"{load_tensor_data_dir}3_{my_batch_size}_aggregated.pt")
        data = torch.cat((tensor_1, tensor_2, tensor_3), dim=0)
        assert len(data) == len(tensor_1) + len(tensor_2) + len(tensor_3)
        print("Full aggregated data loaded.")
    else:
        tensor_1 = torch.load(f"{load_tensor_data_dir}1_{my_batch_size}.pt")
        tensor_2 = torch.load(f"{load_tensor_data_dir}2_{my_batch_size}.pt")
        tensor_3 = torch.load(f"{load_tensor_data_dir}3_{my_batch_size}.pt")
        data = torch.cat((tensor_1, tensor_2, tensor_3), dim=0)
        assert len(data) == len(tensor_1) + len(tensor_2) + len(tensor_3)
        print("Full non-aggregated data loaded.")
else:
    if not aggregated:
        data = torch.load(f"{load_tensor_data_dir}{my_file}_{my_batch_size}.pt")
        print(f"Aggregated data from file {my_file} loaded.")
    else:
        print("Are you sure?")

Full non-aggregated data loaded.


### 1.2 Load the labels

In [10]:
if my_file == "all":
    if aggregated:
        labels_1 = torch.load(f"{load_tensor_labels_dir}1_{my_batch_size}_aggregated.pt")
        labels_2 = torch.load(f"{load_tensor_labels_dir}2_{my_batch_size}_aggregated.pt")
        labels_3 = torch.load(f"{load_tensor_labels_dir}3_{my_batch_size}_aggregated.pt")
        labels = torch.cat((labels_1, labels_2, labels_3), dim=0)
        assert len(labels) == len(labels_1) + len(labels_2) + len(labels_3)
        assert len(labels_1) == len(tensor_1)
        assert len(labels_2) == len(tensor_2)
        assert len(labels_3) == len(tensor_3)
        print("Full aggregated labels loaded.")
    else:
        labels_1 = torch.load(f"{load_tensor_labels_dir}1_{my_batch_size}.pt")
        labels_2 = torch.load(f"{load_tensor_labels_dir}2_{my_batch_size}.pt")
        labels_3 = torch.load(f"{load_tensor_labels_dir}3_{my_batch_size}.pt")
        labels = torch.cat((labels_1, labels_2, labels_3), dim=0)
        assert len(labels) == len(labels_1) + len(labels_2) + len(labels_3)
        assert len(labels_1) == len(tensor_1)
        assert len(labels_2) == len(tensor_2)
        assert len(labels_3) == len(tensor_3)
        print("Full non-aggregated labels loaded.")
else:
    if not aggregated:
        labels = torch.load(f"{load_tensor_labels_dir}{my_file}_{my_batch_size}.pt")
        print(f"Labels from file {my_file} loaded.")
    else:
        print("Are you sure?")

Full non-aggregated labels loaded.


### 1.3 Check shape

In [11]:
data.shape

torch.Size([8177, 75, 1200])

In [12]:
labels.shape

torch.Size([8177])

In [14]:
labels[1234].item()

2

In [19]:
data[1234].shape

torch.Size([75, 1200])

### 1.4 Remove possible empty data in data 1

In [ ]:
# assert my_batch_size == 1

# data_squeezed = data.squeeze(-1)
# print(data_squeezed.shape)

# # find indices
# indices_with_ones = torch.any(data_squeezed == 1, dim=1)

# # filter
# data = data[indices_with_ones]
# labels = labels[indices_with_ones]

# assert len(data) == len(labels)

# 2 Token form dataframe

### 2.1 Temporal

 #### 2.1.1 for each batch, with start and end for each neuron

 ```
 <|start|><|s_0|>00000000011000000010<|e_0|><|s_1>00000000000000010000<|e_1|>...00000100000000111000<|e_74|><|end|>
 ```

In [ ]:
def tensor2df_temporal_dataset_1(tensor_data, tensor_labels, directory, file, batch_size, num_neurons=75):
    all_data = []

    print("Currently converting tensors to dataframes:")
    print(f"Saving to: {directory}")
    print(f"Type: temporal")
    print(f"Batch: {batch_size}")
    print(f"File: {file}")

    # tensor_data has shape [batch_index, neuron_index, sampling_points_in_batch_index]
    for batch in tqdm(range(tensor_data.shape[0]), desc="Processing Batches"):
        batch_data = "<|start|>"

        # there are 75 neurons, for each neuron
        for neuron in range(num_neurons):
            # extract 30(or other batch size) data for this neuron and join them
            neuron_data = ''.join([str(tensor_data[batch, neuron, t].item()) for t in range(batch_size)])
            # add to the batch_data
            batch_data += f"<|s_{neuron}|>{neuron_data}<|e_{neuron}|>"
            # repeat over 75 neurons

        # end the batch_data with the end mark
        batch_data += "<|end|>"  # End marker for the whole batch
        # add this batch's data to the overall data
        all_data.append(batch_data)

    token_form_data_df = pd.DataFrame({'data': all_data})

    # add labels to the result dataframe
    labels = tensor_labels.numpy() if tensor_labels.is_cuda else tensor_labels
    token_form_data_df['labels'] = pd.Series(labels)

    result_df_path_name = f"{directory}data_labels_{file}_{batch_size}.csv"
    token_form_data_df.to_csv(result_df_path_name, index=False)
    print(f"Data for file {file} and batch size {batch_size} are saved to {result_df_path_name}.")

In [ ]:
# tensor2df_temporal_dataset_1(data, labels, data_labels_temporal_dir, my_file, my_batch_size)

NameError: name 'transform_into_token_form_temporal' is not defined

 #### 2.1.2 for each batch, without start and end for each neuron

```
'<|start|>00000000110000010000<|>00000000000100000000<|>.....00000100001001100000<|end|>'
```

In [ ]:
def tensor2df_temporal_dataset_2(tensor_data, tensor_labels, directory, file, batch_size, num_neurons=75):
    all_data = []

    print("Currently converting tensors to dataframes:")
    print(f"Saving to: {directory}")
    print(f"Type: temporal")
    print(f"Batch: {batch_size}")
    print(f"File: {file}")

    # tensor_data has shape [batch_index, neuron_index, sampling_points_in_batch_index]
    for batch in tqdm(range(tensor_data.shape[0]), desc="Processing Batches"):
        batch_data = "<|start|>"

        # there are 75 neurons, for each neuron
        for neuron in range(num_neurons):
            # extract 30(or other batch size) data for this neuron and join them
            neuron_data = ''.join([str(tensor_data[batch, neuron, t].item()) for t in range(batch_size)])
            # add to the batch_data
            if neuron == 0:
                batch_data += neuron_data
            else:
                batch_data += f"<|>{neuron_data}"
            # repeat over 75 neurons

        # end the batch_data with the end mark
        batch_data += "<|end|>"  # End marker for the whole batch
        # add this batch's data to the overall data
        all_data.append(batch_data)

    token_form_data_df = pd.DataFrame({'data': all_data})

    # add labels to the result dataframe
    labels = tensor_labels.numpy() if tensor_labels.is_cuda else tensor_labels
    token_form_data_df['labels'] = pd.Series(labels)

    result_df_path_name = f"{directory}data_labels_{file}_{batch_size}.csv"
    token_form_data_df.to_csv(result_df_path_name, index=False)
    print(f"Data for file {file} and batch size {batch_size} are saved to {result_df_path_name}.")

In [ ]:
tensor2df_temporal_dataset_2(data, labels, data_labels_temporal_dir, my_file, my_batch_size)

Currently converting tensors to dataframes:
Saving to: drive/MyDrive/neuro2voc/task_1/data/3_data_labels_df_temporal/
Type: temporal
Batch: 900
File: all


Processing Batches: 100%|██████████| 6543/6543 [48:30<00:00,  2.25it/s]


Data for file all and batch size 900 are saved to drive/MyDrive/neuro2voc/task_1/data/3_data_labels_df_temporal/data_labels_all_900.csv.


### 2.2 Spatial

If you input data trained with a batch_size of 1, then you should get something like
```
<s>000000000010000000001001010101...010000<e>
```
for each data. You can concatenate it later. Special token: <|>

#### 2.2.1 Normal, with start and end for each batch

In [20]:
def tensor2data_spatial_1(tensor_data, tensor_labels, directory, file, batch_size, num_neurons=75):
    all_data = []
    result_df_path_name = f"{directory}data_labels_{file}_{batch_size}.csv"

    print("Currently converting tensors to dataframes:")
    print(f"Saving to: {result_df_path_name}")
    print(f"Type: spatial, no zeros removed")
    print(f"Batch: {batch_size}")
    print(f"File: {file}")

    # tensor_data has shape [batch_index, neuron_index, sampling_points_in_batch_index]
    for batch in tqdm(range(tensor_data.shape[0]), desc="Processing Batches"):
        batch_data = ""

        # loop thru each time point in the batch
        for t in range(batch_size):
            time_point_data = ''.join([str(tensor_data[batch, neuron, t].item()) for neuron in range(num_neurons)])

            # add the collected data for this time point to the batch data
            batch_data += f"<|start|>{time_point_data}<|end|>" # use these as dividers, add to the special tokens

        # batch_data += "<|end|>"  # End marker for the whole batch
        # add this batch's data to the overall data
        all_data.append(batch_data)

    # create a DataFrame with the data
    token_form_data_df = pd.DataFrame({'data': all_data})

    # add labels to the result DataFrame
    labels = tensor_labels.numpy() if tensor_labels.is_cuda else tensor_labels
    token_form_data_df['labels'] = pd.Series(labels)

    # save the DataFrame to a CSV file

    token_form_data_df.to_csv(result_df_path_name, index=False)
    print(f"Data for file {file} and batch size {batch_size} are saved to {result_df_path_name}.")

In [21]:
tensor2data_spatial_1(data, labels, data_labels_spatial_dir, my_file, my_batch_size)

Currently converting tensors to dataframes:
Saving to: drive/MyDrive/neuro2voc/task_1/data/3_data_labels_df_spatial/data_labels_all_1200.csv
Type: spatial, no zeros removed
Batch: 1200
File: all


Processing Batches: 100%|██████████| 8177/8177 [44:58<00:00,  3.03it/s]


Data for file all and batch size 1200 are saved to drive/MyDrive/neuro2voc/task_1/data/3_data_labels_df_spatial/data_labels_all_1200.csv.


#### 2.2.2 Remove zero entries

Only works for batch_size = 1 data.

In [ ]:
def tensor2data_spatial_2(tensor_data, tensor_labels, directory, file, batch_size=1, num_neurons=75):
    all_data = []
    assert batch_size == 1 # no loop within batch_size
    filtered_labels = []
    result_df_path_name = f"{directory}data_labels_{file}_{batch_size}.csv"

    print("Currently converting tensors to dataframes:")
    print(f"Saving to: {result_df_path_name}")
    print(f"Type: spatial, zeros removed")
    print(f"Batch: {batch_size}")
    print(f"File: {file}")

    # tensor_data has shape [batch_index, neuron_index, sampling_points_in_batch_index]
    for batch in tqdm(range(tensor_data.shape[0]), desc="Processing Batches"):
        # last dimension is always 1 for batch_size 1
        time_point_data = ''.join([str(tensor_data[batch, neuron, 0].item()) for neuron in range(num_neurons)])

        # if the time point data is a string of 75 zeros
        if time_point_data != "0" * num_neurons:
            all_data.append(f"<|start|>{time_point_data}<|end|>")
            # add the corresponding label
            label = tensor_labels[batch].item() if tensor_labels.is_cuda else tensor_labels[batch]
            filtered_labels.append(label)

    # create a DataFrame with the data
    data_labels_df = pd.DataFrame({'data': all_data, 'labels': filtered_labels})

    # Save the DataFrame to a CSV file

    data_labels_df.to_csv(result_df_path_name, index=False)
    print(f"Data for file {file} and batch size {batch_size} are saved to {result_df_path_name}.")

In [ ]:
tensor2data_spatial_2(data, labels, data_labels_spatial_dir, my_file, my_batch_size)

Currently converting tensors to dataframes:
Saving to: drive/MyDrive/neuro2voc/task_1/data/3_data_labels_df_spatial/
Type: spatial
Batch: 1
File: all
Remove zero: Yes


Processing Batches: 100%|██████████| 7639653/7639653 [1:04:37<00:00, 1970.34it/s]


Data for file all and batch size 1 are saved to drive/MyDrive/neuro2voc/task_1/data/3_data_labels_df_spatial/data_labels_all_1.csv.


ArrowInvalid: ('Could not convert tensor(0) with type Tensor: did not recognize Python value type when inferring an Arrow data type', 'Conversion failed for column labels with type object')

### 2.3 Aggregated
Dat is aggregated over the whole batch.


In [ ]:
def tensor2df_aggregated(tensor_data, tensor_labels, directory, file, batch_size, num_neurons=75):
    all_data = []
    result_df_path_name = f"{directory}data_labels_{file}_{batch_size}_aggregated.csv"

    print("Currently converting tensors to dataframes:")
    print(f"Saving to: {result_df_path_name}")
    print(f"Type: spatial, aggregated")
    print(f"Batch size: {batch_size}")
    print(f"File: {file}")

    # tensor_data has shape [batch_index, neuron_index]
    for batch in tqdm(range(tensor_data.shape[0]), desc="Processing Batches"):
        # represent the neuron spikes in this batch as a string
        neuron_spiked = ''.join([str(tensor_data[batch, neuron].item()) for neuron in range(num_neurons)])
        all_data.append(f"<|start|>{neuron_spiked}<|end|>")

    # create a DataFrame with the data
    token_form_data_df = pd.DataFrame({'data': all_data})

    labels = tensor_labels.numpy() if tensor_labels.is_cuda else tensor_labels
    token_form_data_df['labels'] = pd.Series(labels)

    # save df to csv

    token_form_data_df.to_csv(result_df_path_name, index=False)
    print(f"Data for file {file} and batch size {batch_size} are saved to {result_df_path_name}.")

In [ ]:
tensor2df_aggregated(data, labels, data_labels_spatial_dir, my_file, my_batch_size)

Currently converting tensors to dataframes:
Saving to: drive/MyDrive/neuro2voc/task_1/data/3_data_labels_df_spatial/data_labels_all_40_aggregated.csv
Type: spatial, aggregated
Batch size: 40
File: all


Processing Batches:   2%|▏         | 29310/1502308 [00:11<09:34, 2562.08it/s]


KeyboardInterrupt: 

# 3 Data to train tokenizer
because the previous result from section 2 just doesnt work
use data from batch_size 1, do it for once and for all.

### 3.1 For temporal data

In [ ]:
def tensor2hdf5_temporal_tokenizer(tensor_data, directory, file, batch_size=30, num_neurons=75):
    if not os.path.exists(directory):
        os.makedirs(directory)

    hdf5_file_path = os.path.join(directory, f"temporal_{file}_{batch_size}.hdf5")
    # total number
    total_entries = tensor_data.shape[0] * num_neurons

    print("Currently flattening tensors to train tokenizer:")
    print(f"Saving to: {hdf5_file_path}")
    print(f"Type: temporal, non-aggregated")
    print(f"Batch: {batch_size}")
    print(f"File: {file}")

    with h5py.File(hdf5_file_path, 'w') as hdf5_file:
        # create a dataset in HDF5 file
        hdf5_dataset = hdf5_file.create_dataset('data', (total_entries,), dtype=h5py.string_dtype(encoding='utf-8'))

        # process and store each neuron's data individually
        entry_index = 0
        for batch in tqdm(range(tensor_data.shape[0]), desc="Processing Batches"):
            # for each batch, loop thru all neurons
            for neuron in range(num_neurons):
                # for each neuron, extract and join data of batch size 30/60/120
                neuron_data = ''.join([str(tensor_data[batch, neuron, t].item()) for t in range(batch_size)])
                # write directly to HDF5, each neuron data as a separate entry
                hdf5_dataset[entry_index] = neuron_data
                entry_index += 1

    nl = '\n'
    print(f"You can train your tokenizer using data from file {file} and batch size {batch_size}.{nl}It is saved to {hdf5_file_path}.")

In [ ]:
tensor2hdf5_temporal_tokenizer(data, for_tokenizer_dir, my_file, my_batch_size)

Currently flattening tensors to train tokenizer:
Saving to: drive/MyDrive/neuro2voc/task_1/data/3_train_tokenizer_2/
Type: temporal
Batch: 900
File: all


Processing Batches: 100%|██████████| 6543/6543 [52:15<00:00,  2.09it/s]

You can train your tokenizer using data from file all and batch size 900.
It is saved to drive/MyDrive/neuro2voc/task_1/data/3_train_tokenizer_2/temporal_all_900.hdf5.


### 3.2 For spatial data

#### 3.2.1 Normal spatial data

In [ ]:
def tensor2hdf5_spatial_1(tensor_data, directory, file, batch_size=30, num_neurons=75):
    if not os.path.exists(directory):
        os.makedirs(directory)

    hdf5_file_path = os.path.join(directory, f"spatial_{file}_{batch_size}.hdf5")
    # total number
    total_entries = tensor_data.shape[0] * batch_size

    print("Currently flattening tensors to train tokenizer:")
    print(f"Saving to: {hdf5_file_path}")
    print(f"Type: spatial, non-aggregated")
    print(f"Batch: {batch_size}")
    print(f"File: {file}")

    with h5py.File(hdf5_file_path, 'w') as hdf5_file:
        # create a dataset in HDF5 file
        hdf5_dataset = hdf5_file.create_dataset('data', (total_entries), dtype=h5py.string_dtype(encoding='utf-8'))

        # process each batch
        entry_index = 0
        for batch in tqdm(range(tensor_data.shape[0]), desc="Processing Batches"):
            # for each batch, loop thru all timepoints
            for t in range(batch_size):
                time_point_data = ''.join([str(tensor_data[batch, neuron, t].item()) for neuron in range(num_neurons)])
                hdf5_dataset[entry_index] = time_point_data
                entry_index += 1

    nl = '\n'
    print(f"You can train your tokenizer using data from file {file} and batch size {batch_size}.{nl}It is saved to {hdf5_file_path}.")

In [ ]:
tensor2hdf5_spatial_1(data, for_tokenizer_dir, my_file, my_batch_size)

Currently flattening tensors to train tokenizer:
Saving to: drive/MyDrive/neuro2voc/task_1/data/3_train_tokenizer/spatial_all_900.hdf5
Type: spatial, non-aggregated
Batch: 900
File: all


Processing Batches: 100%|██████████| 6543/6543 [41:42<00:00,  2.61it/s]

You can train your tokenizer using data from file all and batch size 900.
It is saved to drive/MyDrive/neuro2voc/task_1/data/3_train_tokenizer/spatial_all_900.hdf5.


#### 3.2.2 Remove zero entries

In [ ]:
def tensor2hdf5_spatial_2_to_hf_dataset(tensor_data, directory, file, batch_size=1, num_neurons=75):
    if not os.path.exists(directory):
        os.makedirs(directory)

    print("Currently flattening tensors to create a Hugging Face dataset.")

    # Prepare data
    data_entries = []
    for batch in tqdm(range(tensor_data.shape[0]), desc="Processing Batches"):
        for t in range(batch_size):
            time_point_data = ''.join([str(tensor_data[batch, neuron, t].item()) for neuron in range(num_neurons)])
            if time_point_data != "0" * num_neurons:
                data_entries.append({'data': time_point_data})

    # Create a Hugging Face Dataset
    hf_dataset = Dataset.from_dict({'data': [entry['data'] for entry in data_entries]})

    # Save the dataset (locally or upload to the Hugging Face Hub)
    hf_dataset_path = os.path.join(directory, f"hf_spatial_{file}_{batch_size}.hfd")
    hf_dataset.save_to_disk(hf_dataset_path)

    print(f"Dataset saved to {hf_dataset_path}.")

    # 之前的不能用因为hdf5 initialize的时候创建了过多empty的。。。


In [ ]:
tensor2hdf5_spatial_2(data, for_tokenizer_dir, my_file, batch_size=1)

Currently flattening tensors to train tokenizer:
Saving to: drive/MyDrive/neuro2voc/task_1/data/3_train_tokenizer/spatial_all_1.hdf5
Type: spatial, non-aggregated, zero-removed
Batch: 1
File: all


Processing Batches: 100%|██████████| 7639653/7639653 [1:07:48<00:00, 1877.88it/s]

You can train your tokenizer using data from file all and batch size 1.
It is saved to drive/MyDrive/neuro2voc/task_1/data/3_train_tokenizer/spatial_all_1.hdf5.


In [ ]:
''.join([str(data[1, neuron, 1].item()) for neuron in range(75)])

IndexError: index 1 is out of bounds for dimension 2 with size 1

### 3.3 For aggregated data

In [ ]:
def tensor2hdf5_spatial_aggregated_tokenizer(tensor_data, directory, file, batch_size, num_neurons=75):
    if not os.path.exists(directory):
        os.makedirs(directory)

    hdf5_file_path = os.path.join(directory, f"spatial_{file}_{batch_size}_aggregated.hdf5")
    # total number of entries is equal to the first dimension of tensor_data
    total_entries = tensor_data.shape[0]

    print("Currently flattening tensors to train tokenizer:")
    print(f"Saving to: {hdf5_file_path}")
    print(f"Type: spatial, aggregated")
    print(f"Batch: {batch_size}")
    print(f"File: {file}")

    with h5py.File(hdf5_file_path, 'w') as hdf5_file:
        # create a dataset in HDF5 file
        hdf5_dataset = hdf5_file.create_dataset('data', (total_entries,), dtype=h5py.string_dtype(encoding='utf-8'))

        # process each batch
        for batch in tqdm(range(tensor_data.shape[0]), desc="Processing Batches"):
            # concatenate neuron data for this batch
            batch_data = ''.join([str(tensor_data[batch, neuron].item()) for neuron in range(num_neurons)])
            hdf5_dataset[batch] = batch_data

    nl = '\n'
    print(f"You can train your tokenizer using data from file {file}.{nl}It is saved to {hdf5_file_path}.")

In [ ]:
tensor2hdf5_spatial_aggregated_tokenizer(data, for_tokenizer_dir, my_file, my_batch_size)

Currently flattening tensors to train tokenizer:
Saving to: drive/MyDrive/neuro2voc/task_1/data/3_train_tokenizer/spatial_all_40_aggregated.hdf5
Type: spatial, aggregated
Batch: 40
File: all


Processing Batches: 100%|██████████| 1502308/1502308 [15:22<00:00, 1627.66it/s]

You can train your tokenizer using data from file all.
It is saved to drive/MyDrive/neuro2voc/task_1/data/3_train_tokenizer/spatial_all_40_aggregated.hdf5.


# 4 Inspect Data

In [ ]:
inspect_file = "all"
inspect_batch_size = 900
# inspect_type = "temporal"
inspect_type = "spatial"

if inspect_type == "temporal":
    inspect_dir = data_labels_temporal_dir
elif inspect_type == "spatial":
    inspect_dir = data_labels_spatial_dir

In [ ]:
if aggregated:
    data_labels_df = pd.read_csv(f"{inspect_dir}data_labels_{inspect_file}_{inspect_batch_size}_aggregated.csv")
else:
    data_labels_df = pd.read_csv(f"{inspect_dir}data_labels_{inspect_file}_{inspect_batch_size}.csv")

In [ ]:
# start symbole: 9, end: 7, intervals in between: <|> 3 * (75 neurons - 1)
if inspect_type == "temporal":
    assert len(data_labels_df['data'][0]) - 9 - 7 - 74*3 == 75*inspect_batch_size
elif inspect_type == "spatial":
    if aggregated:
        assert len(data_labels_df['data'][0]) - 9 - 7 == 75
    else:
        assert len(data_labels_df['data'][0]) - (9 + 7)*inspect_batch_size == 75*inspect_batch_size

In [ ]:
data_labels_df['data'][0]

'<|start|>000000000000000000000000000000000000000000000000000000000000000000000000000<|end|><|start|>000000000000000000000000000000000000000000000000000000000000000000000000000<|end|><|start|>000000000000000000000000000000000000000000000000000000000000000000000000000<|end|><|start|>000000000000000000000000000000000000000000000000000000000000000000000000000<|end|><|start|>000000000000000000000000000000000000000000000000000000000000000000000000000<|end|><|start|>000000000000000000000000000000000000000000000000000000000000000000000000000<|end|><|start|>000000000000000000000000000000000000000000000000000000000000000000000000000<|end|><|start|>000000000000000000000000000000000000000000000000000000000000000000000000000<|end|><|start|>000000000000000000000000000000000001000000000000000000000000000000000000000<|end|><|start|>000000000000000000000000000000000000000000000000000000000000000000000000000<|end|><|start|>000000000000000000000000000000000000000000000000000000000000000000000000000<|end

# Archives

In [ ]:
for_tokenizer_dir = data_dir+"3_train_tokenizer/"

### For each batch
Slice the data by each batch and neuron

Here we don't care if neuron 1 is different from neuron 72. We just need the data for tokenizer.

In [ ]:
reshaped_data_t_30 = combined_data_30.view(-1, combined_data_30.shape[2])
reshaped_data_t_60 = combined_data_60.view(-1, combined_data_60.shape[2])
reshaped_data_t_120 = combined_data_120.view(-1, combined_data_120.shape[2])

In [ ]:
reshaped_data_t_30.shape

torch.Size([18953550, 30])

In [ ]:
torch.save(reshaped_data_t_30, for_tokenizer_dir+'temporal_30.pt')
torch.save(reshaped_data_t_60, for_tokenizer_dir+'temporal_60.pt')
torch.save(reshaped_data_t_120, for_tokenizer_dir+'temporal_120.pt')

### For each timepoint
Slice the data by each sampling time

In [ ]:
reshaped_data_s_30 = combined_data_30.view(-1, combined_data_30.shape[1])
reshaped_data_s_60 = combined_data_60.view(-1, combined_data_60.shape[1])
reshaped_data_s_120 = combined_data_120.view(-1, combined_data_120.shape[1])

In [ ]:
reshaped_data_s_30.shape

torch.Size([7581420, 75])

In [ ]:
reshaped_data_s_30[1]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0], dtype=torch.int32)

In [ ]:
torch.save(reshaped_data_s_30, for_tokenizer_dir+'spatial_30.pt')
torch.save(reshaped_data_s_60, for_tokenizer_dir+'spatial_60.pt')
torch.save(reshaped_data_s_120, for_tokenizer_dir+'spatial_120.pt')